# Python dependencies

In [ ]:
%pip install requests
%pip install pandas

# Configuration

Settings to be configured per individual. 

TODO: configure these settings outside of the notebook so they don't mess with source control. (environment variables?)

In [ ]:
# get an auth token using the steps here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token
# set it in this cell, then delete it to avoid accidentally committing it
authtoken = ''

In [ ]:
import os

resultsDownloadLocation = 'c:\\temp\\testResults'
if (not os.path.exists(resultsDownloadLocation)):
    os.makedirs(resultsDownloadLocation)


# Retrieving Data

The github action "Aggregate Test Results" runs daily and collects all the results for the previous day into a single json file.

These steps will:

- Find the last 50 runs (you can increase this if you want to look back further
- Download the artifacts from those runs into memory
- Write the .json file from within the artifact to disk (only if there isn't already an up to date file on disk)
- Load all results into a pandas DataFrame

In [24]:
import requests

def getRuns():
    runsResponse = requests.get(
        "https://api.github.com/repos/microsoft/vscode-jupyter/actions/workflows/aggregate-test-results.yml/runs?per_page=50",
        headers={
            "Accept": "application/vnd.github+json",
            "Authorization": f"Bearer {authtoken}",
            },   
    )
    
    if runsResponse.status_code != 200:
        print(f"Error {runsResponse.status_code}")
        raise Exception("Error getting runs")

    print(f"Found {len(runsResponse.json()['workflow_runs'])} runs")

    return runsResponse.json()["workflow_runs"]

runs = getRuns()

Found 30 runs


In [25]:
from datetime import datetime

alreadyDownloaded = {}
for file in os.listdir(resultsDownloadLocation):
    path = os.path.join(resultsDownloadLocation, file)
    lastModified = datetime.fromtimestamp(os.path.getmtime(path))
    alreadyDownloaded[file] = lastModified

print(f"Already downloaded {len(alreadyDownloaded)} result files, they will be skipped unless there is a newer version")

def shouldDownload(name, timestamp):
    fileDate = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
    if name in alreadyDownloaded:
        if alreadyDownloaded[name] >= fileDate:
            return False
            
    alreadyDownloaded[name] = fileDate
    return True
    

Already downloaded 13 result files, they will be skipped unless there is a newer version


In [26]:
import zipfile
import json
import io

def getArtifactData(id):
    testResultsResponse = requests.get(
        f"https://api.github.com/repos/microsoft/vscode-jupyter/actions/artifacts/{id}/zip",
        headers={
            "Accept": "application/vnd.github+json",
            "Authorization": f"Bearer {authtoken}",
        },
    )

    if testResultsResponse.status_code != 200:
        print(f"Error {testResultsResponse.status_code} getting artifact {id}")

    return testResultsResponse.content

def saveResultsFile(zipData, timeStamp):
    with zipfile.ZipFile(io.BytesIO(zipData)) as artifact:
        for name in artifact.namelist():
            print(f'checking {name} at {timeStamp}')
            if shouldDownload(name, timeStamp):
                content = artifact.read(name)
                print(f"    saving {name}")
                with open(f'{resultsDownloadLocation}\\{name}', 'wb') as f:
                    f.write(content) 

print(f"Getting artifacts from {len(runs)} runs")
for run in runs:
    artifactUrl = run["artifacts_url"]
    print(f"Getting artifacts from {artifactUrl} from {run['created_at']}")
    artifactsResponse = requests.get(
        artifactUrl, headers={
            "Accept": "application/vnd.github+json",
            "Authorization": f"Bearer {authtoken}",
            }
    )

    if artifactsResponse.status_code != 200:
        print(f"Error {artifactsResponse.status_code} getting artifact {id}")
    else:
        artifacts = artifactsResponse.json()["artifacts"]
        for artifact in artifacts:
            rawData = getArtifactData(artifact["id"])
            testRunResults = saveResultsFile(rawData, run["created_at"])

Getting artifacts from 30 runs
Getting artifacts from https://api.github.com/repos/microsoft/vscode-jupyter/actions/runs/3063531278/artifacts from 2022-09-15T20:35:00Z
checking AggTestResults-2022-09-09.json at 2022-09-15T20:35:00Z
    saving AggTestResults-2022-09-09.json
Getting artifacts from https://api.github.com/repos/microsoft/vscode-jupyter/actions/runs/3063528114/artifacts from 2022-09-15T20:34:26Z
checking AggTestResults-2022-09-08.json at 2022-09-15T20:34:26Z
    saving AggTestResults-2022-09-08.json
Getting artifacts from https://api.github.com/repos/microsoft/vscode-jupyter/actions/runs/3063526631/artifacts from 2022-09-15T20:34:10Z
checking AggTestResults-2022-09-07.json at 2022-09-15T20:34:10Z
    saving AggTestResults-2022-09-07.json
Getting artifacts from https://api.github.com/repos/microsoft/vscode-jupyter/actions/runs/3063525414/artifacts from 2022-09-15T20:33:55Z
checking AggTestResults-2022-09-06.json at 2022-09-15T20:33:55Z
    saving AggTestResults-2022-09-06.js

In [33]:
import pandas as pd
from datetime import timedelta

testResults = []
for file in os.listdir(resultsDownloadLocation):
    if datetime.fromtimestamp(os.path.getmtime(path)) < datetime.now() - timedelta(days=50):
        # limit the amount of results we load
        continue

    with open(f'{resultsDownloadLocation}\\{file}', 'r') as f:
        try:
            df = pd.read_json(f)
            testResults.append(df)
        except Exception as e:
            print(f'Error reading {file}: {e}')

df = pd.concat(testResults)
# strip off the time to help grouping, but keep as datetime type
df["datetime"] = pd.to_datetime(df["date"])
df["date"] = pd.to_datetime(df["date"]).dt.date

print(f"{len(df)} test results collected between {df['date'].min()} and {df['date'].max()}")

df.head()

140839 test results collected between 2022-08-31 and 2022-09-14


,scenario,suite,testName,date,runUrl,status,datetime
0,TestResult-local-conda-3.9---ubuntu-latest,3rd Party Kernel Service API,List kernel specs,2022-08-31,https://github.com/microsoft/vscode-jupyter/ac...,passed,2022-08-31 21:59:56+00:00
1,TestResult-local-conda-3.9---ubuntu-latest,3rd Party Kernel Service API,Access Kernels,2022-08-31,https://github.com/microsoft/vscode-jupyter/ac...,passed,2022-08-31 21:59:56+00:00
2,TestResult-local-conda-3.9---ubuntu-latest,3rd Party Kernel Service API,Start Kernel,2022-08-31,https://github.com/microsoft/vscode-jupyter/ac...,passed,2022-08-31 21:59:56+00:00
3,TestResult-local-conda-3.9---ubuntu-latest,Configuration Service,Ensure same instance of settings return,2022-08-31,https://github.com/microsoft/vscode-jupyter/ac...,passed,2022-08-31 21:59:56+00:00
4,TestResult-local-conda-3.9---ubuntu-latest,Configuration Service,Ensure async registry returns expected class,2022-08-31,https://github.com/microsoft/vscode-jupyter/ac...,passed,2022-08-31 21:59:56+00:00


# Reporting

In [28]:
from datetime import date, timedelta
recentFailures = df[df['date'] > date.today() - timedelta(days=7)]
recentFailures = recentFailures[recentFailures['status'] == 'failed'].dropna()
recentFailures = recentFailures.groupby(['testName']).agg(testName_count=('testName', 'count'))
recentFaiulres = recentFailures.rename(columns={'testName_count': 'failureCount'}, inplace=True)

recentFailures.sort_values(by=['failureCount'], ascending=False).head(20)

,failureCount
testName,
Raising an exception from system code has a stack trace,3
Running a cell with markdown and code runs two cells,3
Raising an exception from within a function has a stack trace,3
Cells from python files and the input box are executed in correct order,3
Export Interactive window to Python file,3
Error stack traces have correct line hrefs with mix of cell sources,3
Run current file in interactive window (with cells),2
Multiple interactive windows,2
Run current file in interactive window (without cells),2


In [31]:
testName= 'Cells from python files and the input box are executed in correct order'

testData = df.where(df['testName'] == testName).dropna()
passes = testData.where(testData['status'] == 'passed').dropna()
fails = testData.where(testData['status'] == 'failed').dropna()
successRate = len(passes) / (len(passes) + len(fails))
print(f"'{testName}' failed {len(fails)} times between {testData['date'].min()} and {testData['date'].max()}")
print(f"Success rate: {successRate}")

testData['fail'] = testData['status'] == 'failed'
testData['pass'] = testData['status'] == 'passed'

passfailcounts = testData.groupby(['date']).sum()

passfailcounts.sort_values(by=['date'], ascending=False)

# line chart not working
# import matplotlib.pyplot as plt
# ax=testData.plot(kind='line', x='date', y='pass', color='Green')

# ax2=testData.plot(kind='line', x='date', y='fail', secondary_y=True,color='Red', ax=ax)

# ax.set_ylabel('Passes')
# ax2.set_ylabel('Failures')
# plt.tight_layout()
# plt.show()

'Cells from python files and the input box are executed in correct order' failed 8 times between 2022-08-31 and 2022-09-14
Success rate: 0.9845559845559846


,fail,pass
date,,
2022-09-14,0,9
2022-09-13,0,39
2022-09-12,3,66
2022-09-09,0,9
2022-09-08,0,30
2022-09-07,1,45
2022-09-06,1,83
2022-09-02,1,56
2022-09-01,1,67


In [32]:
failures = testData.where(testData['status'] == 'failed').dropna()
failures = failures[['date', 'status', 'scenario', 'runUrl']].sort_values(by=['date'], ascending=False).head(10)

for index, row in failures.iterrows():
    print(f"{row['date']} - {row['scenario']}\n{row['runUrl']}")

2022-09-12 - TestResult-remote-nonConda-3.9--web-ubuntu-latest
https://github.com/microsoft/vscode-jupyter/actions/runs/3039654787
2022-09-12 - TestResult-remote-nonConda-3.9--web-ubuntu-latest
https://github.com/microsoft/vscode-jupyter/actions/runs/3039141606
2022-09-12 - TestResult-remote-nonConda-3.9--web-ubuntu-latest
https://github.com/microsoft/vscode-jupyter/actions/runs/3038467661
2022-09-07 - TestResult-remote-nonConda-3.9--web-ubuntu-latest
https://github.com/microsoft/vscode-jupyter/actions/runs/3010735863
2022-09-06 - TestResult-remote-nonConda-3.9--web-ubuntu-latest
https://github.com/microsoft/vscode-jupyter/actions/runs/3003869034
2022-09-02 - TestResult-remote-nonConda-3.9--web-ubuntu-latest
https://github.com/microsoft/vscode-jupyter/actions/runs/2981854984
2022-09-01 - TestResult-remote-nonConda-3.9--web-ubuntu-latest
https://github.com/microsoft/vscode-jupyter/actions/runs/2975744539
2022-08-31 - TestResult-remote-nonConda-3.9--web-ubuntu-latest
https://github.com/m